In [1]:
import itertools
import string
from typing import (Any, Callable, Dict, FrozenSet, Hashable, ItemsView, Iterator, TextIO,
                    KeysView, List, Mapping, NamedTuple, Optional,
                    Sequence, Tuple, Union, ValuesView, cast)

import numpy as np

from cirq import ops, abc, value, extension, linalg, circuits
import cirq

In [2]:
def ops_to_matrix(*ops):
    c = cirq.Circuit.from_ops(*ops)
    return c.to_unitary_matrix()
q0 = cirq.NamedQubit('q0')
q1 = cirq.NamedQubit('q1')
q2 = cirq.NamedQubit('q2')
q3 = cirq.NamedQubit('q3')
q4 = cirq.NamedQubit('q4')
q5 = cirq.NamedQubit('q5')

In [15]:
cirq.ops.QubitOrder

cirq.ops.qubit_order.QubitOrder

In [3]:
class DummyOperation(cirq.Operation, cirq.QasmConvertableOperation, cirq.CompositeOperation):
    @property
    def qubits(self):
        return (q0,)
    
    def with_qubits(self, *qubits):
        raise NotImplementedError

    def known_qasm_output(self, args):
        return '// Dummy operation\n'

    def default_decompose(self):
        return ()
    
class DummyCompositeOperation(cirq.Operation, cirq.CompositeOperation):
    @property
    def qubits(self):
        return (q0,)
    
    def with_qubits(self, *qubits):
        raise NotImplementedError

    def default_decompose(self):
        return cirq.X(self.qubits[0])
    
    def __repr__(self):
        return 'DummyCompositeOperation()'

circuit = cirq.Circuit.from_ops(
        cirq.Z(q0),
        cirq.Z(q0) ** .625,
        cirq.Y(q0),
        cirq.Y(q0) ** .375,
        cirq.X(q0),
        cirq.X(q0) ** .875,
        cirq.H(q1),
        cirq.CZ(q0, q1),
        cirq.CZ(q0, q1) ** 0.25,  # Requires 2-qubit decomposition
        cirq.CNOT(q0, q1),
        cirq.CNOT(q0, q1) ** 0.5,  # Requires 2-qubit decomposition
        cirq.SWAP(q0, q1),
        cirq.SWAP(q0, q1) ** 0.75,  # Requires 2-qubit decomposition

        cirq.CCZ(q0, q1, q2),
        cirq.CCX(q0, q1, q2),
        cirq.CSWAP(q0, q1, q2),

        cirq.ISWAP(q2, q0),  # Requires 2-qubit decomposition

        cirq.google.ExpZGate()(q3),
        cirq.google.ExpZGate(half_turns=0.75)(q3),
        cirq.google.ExpWGate(axis_half_turns=0.125, half_turns=0.25)(q1),
        cirq.google.Exp11Gate()(q0, q1),
        # Requires 2-qubit decomposition
        cirq.google.Exp11Gate(half_turns=1.25)(q0, q1),

        #cirq.MeasurementGate('xX')(q0),
        cirq.MeasurementGate('x_a')(q2),
        cirq.MeasurementGate('x?')(q1),
        cirq.MeasurementGate('X')(q3),
        cirq.MeasurementGate('_x')(q4),
        #cirq.MeasurementGate('x_a')(q2),
        #cirq.MeasurementGate('multi', (False, True))(q1, q2, q3),

        DummyOperation(),
        DummyCompositeOperation(),
    )
c = circuit
display(c)
u = c.to_unitary_matrix(qubit_order=[q4, q3, q2, q1, q0])
#np.set_printoptions(precision=3, suppress=5)
#u * np.exp(-1j*np.angle(u[0,0]))

q0: ───Z───Z^0.625───Y───Y^0.375───X───X^0.875───@───@────────@───@───────×───×────────@───@───@───iSwap───────────────────@───@──────────<__main__.DummyOperation object at 0x7f08778e1438>───DummyCompositeOperation()───
                                                 │   │        │   │       │   │        │   │   │   │                       │   │
q1: ───────────────────────────────────H─────────@───@^0.25───X───X^0.5───×───×^0.75───@───@───×───┼───────W(0.125)^0.25───@───@^-0.75────M('x?')──────────────────────────────────────────────────────────────────────────
                                                                                       │   │   │   │
q2: ───────────────────────────────────────────────────────────────────────────────────@───X───×───iSwap───────────────────────M('x_a')────────────────────────────────────────────────────────────────────────────────────

q3: ───────────────────────────────────────────────────────────────────────────────────────────────Z───────Z^0.75─────────────────────────M('X')───────────────────────────────────────────────────────────────────────────

q4: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────M('_x')──────────────────────────────────────────────────────────────────────────

In [4]:
s = c.to_qasm(precision=5)
print(s)

// Generated from Cirq

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [q0, q1, q2, q3, q4]
qreg q[5];
creg m_x_a[1];
creg m0[1];  // Measurement: x?
creg m_X[1];
creg m__x[1];


z q[0];
rz(pi*0.625) q[0];
y q[0];
ry(pi*0.375) q[0];
x q[0];
rx(pi*0.875) q[0];
h q[1];
cz q[0],q[1];

// Gate: CZ**0.25
u3(pi*0.5,pi*1.0,pi*0.75) q[0];
u3(pi*0.5,pi*1.0,pi*0.25) q[1];
rx(pi*0.5) q[0];
cx q[0],q[1];
rx(pi*0.375) q[0];
ry(pi*0.5) q[1];
cx q[1],q[0];
rx(pi*-0.5) q[1];
rz(pi*0.5) q[1];
cx q[0],q[1];
u3(pi*0.5,pi*0.375,0) q[0];
u3(pi*0.5,pi*0.875,0) q[1];

cx q[0],q[1];

// Gate: CNOT**0.5
ry(pi*-0.5) q[1];
u3(pi*0.5,0,pi*0.25) q[0];
u3(pi*0.5,0,pi*0.75) q[1];
rx(pi*0.5) q[0];
cx q[0],q[1];
rx(pi*0.25) q[0];
ry(pi*0.5) q[1];
cx q[1],q[0];
rx(pi*-0.5) q[1];
rz(pi*0.5) q[1];
cx q[0],q[1];
u3(pi*0.5,pi*1.0,pi*1.0) q[0];
u3(pi*0.5,pi*0.5,pi*1.0) q[1];
ry(pi*0.5) q[1];

swap q[0],q[1];

// Gate: SWAP**0.75
cx q[0],q[1];
ry(pi*-0.5) q[0];
u3(pi*0.5,0,pi*0.45919) q[1];
u3(pi*0.5,0,pi*1.95919) q[0];
rx

In [5]:
from qiskit import qasm
import qiskit

f = qasm.Qasm(data=s)
pr = f.parse()
text = pr.qasm()

p = qiskit.QuantumProgram()
p.load_qasm_text(s)
sim_result = p.execute(backend='local_unitary_simulator')
u_qiskit = sim_result.get_unitary()

/usr/local/google/home/cduck/docs/cirqDev/lib/python3.5/site-packages/qiskit/_quantumprogram.py:433: DeprecationWarning: QuantumProgram.load_qasm_text() will be deprecated in upcoming versions (>0.5.0). Using qiskit.load_qasm_text() instead is recommended.
  "Using qiskit.load_qasm_text() instead is recommended.", DeprecationWarning)


In [6]:
cirq.testing.assert_allclose_up_to_global_phase(u, u_qiskit, rtol=1e-10, atol=1e-10)

In [7]:
u

array([[ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [-0.23533543+0.18114224j,  0.16902423+0.39981017j,
        -0.40167231+0.18285096j, ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       ...,
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.10583981-0.0597131j ,
         0.03103786-0.14744755j,  0.07093688-0.03595775j],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        , ...,  0.0732233 -0.03962817j,
        -0.07493202+0.28103786j, -0.09567086-0.1767767j ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j       

In [8]:
u_qiskit

array([[ 1.02811684e-16-3.82234437e-18j, -1.16287782e-16-3.72446058e-18j,
        -7.68435283e-17-3.29568587e-17j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-2.81359628e-01-9.50370208e-02j, -2.38525175e-01+3.62661141e-01j,
        -3.75192198e-01-2.32391773e-01j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       [-3.54501184e-17+3.83538310e-17j, -5.26992817e-18-1.64742234e-16j,
        -8.49682578e-17+1.06857465e-16j, ...,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j],
       ...,
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j, ...,
         1.08451082e-01+5.48277666e-02j,  1.39841871e-01-5.61104340e-02j,
         6.93081992e-02+3.90048027e-02j],
       [ 0.00000000e+00+0.00000000e+00j,  0.

In [9]:
np.set_printoptions(precision=4, suppress=6)

In [10]:
u = ops_to_matrix(
    cirq.google.ExpWGate(axis_half_turns=0.1)(q0),
    cirq.google.ExpWGate(axis_half_turns=0.3, half_turns=0.4)(q0),
)
u * np.exp(-1j*np.angle(u[0,0]))

array([[ 0.5878-0.j    , -0.25  +0.7694j],
       [-0.6545+0.4755j,  0.1816+0.559j ]])

In [11]:
u = ops_to_matrix(
    cirq.google.ExpWGate(axis_half_turns=(0.3-0.2), half_turns=0.4)(q0),
    cirq.google.ExpWGate(axis_half_turns=0.1)(q0),
)
u * np.exp(-1j*np.angle(u[0,0]))

array([[ 0.5878+0.j    ,  0.25  +0.7694j],
       [-0.25  +0.7694j,  0.5878+0.j    ]])

In [12]:
u = ops_to_matrix(
    cirq.google.ExpWGate(axis_half_turns=(0.2-0.3), half_turns=0.4)(q0),
    cirq.google.ExpWGate(axis_half_turns=0.1)(q0),
)
u * np.exp(-1j*np.angle(u[0,0]))

array([[ 0.5878+0.j    , -0.25  +0.7694j],
       [-0.6545+0.4755j,  0.1816+0.559j ]])

In [13]:
cirq.measure_each(q0, q1)

[GateOperation(MeasurementGate('q0', ()), (NamedQubit('q0'),)),
 GateOperation(MeasurementGate('q1', ()), (NamedQubit('q1'),))]